<a href="https://colab.research.google.com/github/Diljitsingh14/AIML/blob/main/SMOL/smol_ex_1_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### SMOL course exercize with bonus stuff.

In [ ]:
# install packages
!pip install trl
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
# Huggingface login using colab secrets [make sure you will add secret into notebook]
from huggingface_hub import login
from google.colab import userdata


login(token=userdata.get('hf'))

In [ ]:
# Import libraries.

from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import setup_chat_format
import torch

In [ ]:
# Load Model [smolLM2-135]

# Dynamically set the device
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model,tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)
print(f"Model loaded on {device}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

Model loaded on cuda


In [3]:
# Define messages for SmolLM2
messages = [
    {"role": "user", "content": "Hello, how are you?"},
    {
        "role": "assistant",
        "content": "I'm doing well, thank you! How can I assist you today?",
    },
    {"role": "user", "content": "I'd like to show off how chat templating works!"},
]

In [ ]:
input_text = tokenizer.apply_chat_template(messages, tokenize=False)
input_text

"<|im_start|>user\nHello, how are you?<|im_end|>\n<|im_start|>assistant\nI'm doing well, thank you! How can I assist you today?<|im_end|>\n<|im_start|>user\nI'd like to show off how chat templating works!<|im_end|>\n"

In [ ]:
## By default tokenize=True which return the tokens

input_text = tokenizer.apply_chat_template(
    messages,  add_generation_prompt=True
)

print("Conversation decoded:", tokenizer.decode(token_ids=input_text))

Conversation decoded: <|im_start|>user
Hello, how are you?<|im_end|>
<|im_start|>assistant
I'm doing well, thank you! How can I assist you today?<|im_end|>
<|im_start|>user
I'd like to show off how chat templating works!<|im_end|>
<|im_start|>assistant



In [ ]:
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

print("Conversation tokenized:", input_text)

Conversation tokenized: [1, 4093, 198, 19556, 28, 638, 359, 346, 47, 2, 198, 1, 520, 9531, 198, 57, 5248, 2567, 876, 28, 9984, 346, 17, 1073, 416, 339, 4237, 346, 1834, 47, 2, 198, 1, 4093, 198, 57, 6737, 702, 288, 1138, 767, 638, 11743, 8485, 674, 1806, 17, 2, 198, 1, 520, 9531, 198]


In [ ]:
# load dataset [smoltalk/everyday]
from datasets import load_dataset

ds = load_dataset("HuggingFaceTB/smoltalk", "everyday-conversations")

README.md:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/946k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/52.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2260 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/119 [00:00<?, ? examples/s]

In [ ]:
## No need to format in role and content message keys for chat format because smol dataset already formatted in that way.

def process_dataset(sample):
    sample['text'] =  tokenizer.apply_chat_template(sample['messages'], tokenize=False)
    return sample

In [ ]:
# tokenizer.apply_chat_template(ds['train'][0]['messages'], tokenize=False)
ds = ds.map(process_dataset)

In [ ]:
ds['train'][1]['text']

"<|im_start|>user\nHi<|im_end|>\n<|im_start|>assistant\nHello! How can I help you today?<|im_end|>\n<|im_start|>user\nI'm looking for career advice. I want to find a new job, but I'm not sure what I want to do.<|im_end|>\n<|im_start|>assistant\nCareer development can be challenging. What are your current skills and interests that might help narrow down some options?<|im_end|>\n<|im_start|>user\nI have experience in marketing and enjoy working with people. I'm also interested in learning more about data analysis.<|im_end|>\n<|im_start|>assistant\nThat's a great combination. You might consider roles like marketing analyst or business development, which combine people skills with data analysis. I can provide more information on those careers if you'd like.<|im_end|>\n<|im_start|>user\nThat sounds helpful. Can you also suggest some resources for learning data analysis?<|im_end|>\n<|im_start|>assistant\nAbsolutely. Online courses like Coursera, edX, and LinkedIn Learning offer a wide range 

#### Lets Try with different dataset gsm8k is in different format so we need to make in chat format and then tokenize it.

In [ ]:
# openai/gsm8k
ds_openai_gsm8k = load_dataset( "openai/gsm8k","main")


train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [ ]:
## Let See the format.
ds_openai_gsm8k['train'][0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}

In [ ]:
def process_dataset(sample):
    message = [
        {"role": "user", "content": sample['question']},
        {"role": "assistant", "content": sample['answer']},
    ]
    sample['text'] =  tokenizer.apply_chat_template(message, tokenize=False)
    return sample

ds_openai_gsm8k.map(process_dataset)

NameError: name 'ds_openai_gsm8k' is not defined

#### BONUS


*   Pipeline
*   List item





In [6]:
from transformers import pipeline
import torch

pipe = pipeline("text-generation", "meta-llama/Meta-Llama-3-8B-Instruct", torch_dtype=torch.bfloat16, device_map="auto")
response = pipe(messages, max_new_tokens=128)
response

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': [{'role': 'user', 'content': 'Hello, how are you?'},
   {'role': 'assistant',
    'content': "I'm doing well, thank you! How can I assist you today?"},
   {'role': 'user',
    'content': "I'd like to show off how chat templating works!"},
   {'role': 'assistant',
    'content': "That sounds like a lot of fun! I'm happy to play along. Go ahead and show me what you've got."}]}]

In [7]:
chat = response[0]['generated_text']
chat.append(
    {"role": "user", "content": "write some code to show some examples how to you it."}
)
response = pipe(chat, max_new_tokens=528)
response

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': [{'role': 'user', 'content': 'Hello, how are you?'},
   {'role': 'assistant',
    'content': "I'm doing well, thank you! How can I assist you today?"},
   {'role': 'user',
    'content': "I'd like to show off how chat templating works!"},
   {'role': 'assistant',
    'content': "That sounds like a lot of fun! I'm happy to play along. Go ahead and show me what you've got."},
   {'role': 'user',
    'content': 'write some code to show some examples how to you it.'},
   {'role': 'assistant',
    'content': 'Let\'s use a simple templating language like Handlebars. Here\'s an example code:\n```\n// Define a template\nvar template = Handlebars.compile(\'{{name}} is {{age}} years old.\');\n\n// Define some data\nvar data = {\n  name: \'John\',\n  age: 30\n};\n\n// Render the template with the data\nvar result = template(data);\n\nconsole.log(result); // Output: "John is 30 years old."\n```\nIn this example, the template has two placeholders: `{{name}}` and `{{age}}`. When 

In [9]:
print(response[0]['generated_text'][-1]['content'])

Let's use a simple templating language like Handlebars. Here's an example code:
```
// Define a template
var template = Handlebars.compile('{{name}} is {{age}} years old.');

// Define some data
var data = {
  name: 'John',
  age: 30
};

// Render the template with the data
var result = template(data);

console.log(result); // Output: "John is 30 years old."
```
In this example, the template has two placeholders: `{{name}}` and `{{age}}`. When we render the template with the `data` object, Handlebars replaces the placeholders with the corresponding values from the data object.

Here are a few more examples:

**Example 2: Conditionals**
```
var template = Handlebars.compile('{{#if isAdmin}}You are an admin{{/if}}');
var data = {
  isAdmin: true
};
var result = template(data);
console.log(result); // Output: "You are an admin"

// If isAdmin is false, the output would be empty
```
**Example 3: Loops**
```
var template = Handlebars.compile('{{#each friends}} {{name}} is a friend{{/each}}'

In [19]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# model = AutoModelForCausalLM("meta-llama/Meta-Llama-3-8B-Instruct", torch_dtype=torch.bfloat16,device_map="auto")
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

model = pipe.model

tokenizer = pipe.tokenizer

chat = [
    {"role": "system", "content": "You are a sassy, wise-cracking robot as imagined by Hollywood circa 1986."},
    {"role": "user", "content": "Hey, can you tell me any fun things to do in New York?"}
]

formatted_chat = tokenizer.apply_chat_template(chat, tokenize=False)
inputs = tokenizer(formatted_chat,add_special_tokens=False, return_tensors="pt")

inputs = {key: tensor.to(pipe.device) for key, tensor in inputs.items()}
print("Tokenized inputs:\n", inputs)

outputs = model.generate(**inputs,max_new_tokens=128,temperature=0.1)
print("Generated tokens:\n", outputs)

decoded_output = tokenizer.decode(outputs[0][inputs['input_ids'].size(1):], skip_special_tokens=True)
print("Decoded output:\n", decoded_output)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Tokenized inputs:
 {'input_ids': tensor([[128000, 128006,   9125, 128007,    271,   2675,    527,    264,    274,
          27801,     11,  24219,  48689,   9162,  12585,    439,  35706,    555,
          17681,  54607,    220,   3753,     21,     13, 128009, 128006,    882,
         128007,    271,  19182,     11,    649,    499,   3371,    757,    904,
           2523,   2574,    311,    656,    304,   1561,   4356,     30, 128009]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}
Generated tokens:
 tensor([[128000, 128006,   9125, 128007,    271,   2675,    527,    264,    274,
          27801,     11,  24219,  48689,   9162,  12585,    439,  35706,    555,
          17681,  54607,    220,   3753,     21,     13, 128009, 128006,    882,
         128007,    271,  19182,     11,    649,    499,   3371,    757,   

#### Qauntization
It is possible to go even lower than 16-bits using “quantization”, a method to lossily compress model weights. This allows each parameter to be squeezed down to 8 bits, 4 bits or even less. Note that, especially at 4 bits, the model’s outputs may be negatively affected, but often this is a tradeoff worth making to fit a larger and more capable chat model in memory. Let’s see this in action with bitsandbytes:

In [26]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)


quantization_config = BitsAndBytesConfig(load_in_4bit=True)  # You can also try load_in_4bit
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quantization_config)
inputs = tokenizer("Hello how are you?", return_tensors="pt",add_special_tokens=False)
inputs = {key: tensor.to(model.device) for key,tensor in inputs.items()}
res = model.generate(**inputs,max_new_tokens=128)
print(tokenizer.decode(res[0]))

ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`